In [38]:
import pandas as pd
import numpy as np
import sqlite3
db = sqlite3.connect('Recitation')
c= db.cursor()

In [39]:
pd.read_sql_query('select * from Student',con=db)

,sid,name
0,1,Jill
1,2,Bo
2,3,Maya


In [40]:
pd.read_sql_query('select * from Course',con=db)

,cid,sem,subj
0,550,S15,DB
1,677,S15,Algo
2,521,F15,AI
3,700,S17,BigData


In [41]:
pd.read_sql_query('select * from Professor',con=db)

,fid,name
0,11,Ives
1,12,Khanna
2,15,Davidson
3,18,Marcus


In [42]:
pd.read_sql_query('select * from Takes',con=db)

,sid,cid,sem,grade
0,1,550,S17,A
1,1,677,S15,A
2,3,550,S15,C
3,3,521,F15,C
4,3,700,S17,A


In [43]:
pd.read_sql_query('select * from Teaches',con=db)

,fid,cid,sem
0,11,550,S15
1,12,677,S15
2,18,521,F15
3,11,700,S17
4,15,550,F16


In [44]:
#Names of all students
pd.read_sql_query('select distinct name from Student',con=db)

,name
0,Bo
1,Jill
2,Maya


In [45]:
#Semester in which 550 is offered
pd.read_sql_query('select distinct sem from Takes where cid=\"550\";',con=db)

,sem
0,S17
1,S15


In [46]:
#Names of professors who teach in F15.
stmt= ('select Name '+
       'from Teaches as t join Professor as p on t.fid=p.fid ' +
       'where sem=\"F15\";')
pd.read_sql_query(stmt,con=db)

,name
0,Marcus


In [47]:
#Names of professors who teach in F15.
stmt= ('select Name '+
       'from Teaches natural join Professor ' +
       'where sem=\"F15\";')
pd.read_sql_query(stmt,con=db)

,name
0,Marcus


In [48]:
#IDs of courses taken by Jill
stmt= ('select cid '+
       'from Takes natural join Student ' +
       'where name=\"Jill\";')
pd.read_sql_query(stmt,con=db)

,cid
0,550
1,677


In [49]:
#Ids of courses for students getting a C.
stmt= (' select distinct cid ' +
       ' from Takes ' +
       ' where grade=\"C\";')
pd.read_sql_query(stmt,con=db)

,cid
0,550
1,521


In [50]:
#Names and sid of students who have taken more than one course from the same professor.
stmt= ('select distinct t1.sid as sid, name ' +
       'from (Takes natural join Teaches) as t1, ' +
        '    (Takes natural join Teaches) as t2, ' +   
        '     Student as s ' +
        'where t1.sid = t2.sid and t1.fid=t2.fid and t1.cid<>t2.cid and s.sid=t1.sid;' )
pd.read_sql_query(stmt,con=db)

,sid,name
0,3,Maya


In [51]:
#Ordering
stmt= ('select * ' +
       'from Course '+
       'order by subj, sem; ')
pd.read_sql_query(stmt,con=db)

,cid,sem,subj
0,521,F15,AI
1,677,S15,Algo
2,700,S17,BigData
3,550,S15,DB


In [52]:
#Ids of students who will get at least A and one C
stmt= ('select sid ' +
       ' from Takes ' +
       ' where grade=\"A\" ' +
       ' INTERSECT ' +
        'select sid ' +
       ' from Takes ' +
       ' where grade=\"C\";')
pd.read_sql_query(stmt,con=db)

,sid
0,3


In [53]:
#Ids of students who will get an A or a C
stmt= ('select distinct sid ' +
       ' from Takes ' +
       ' where grade=\"A\" or grade=\"C\";')
pd.read_sql_query(stmt,con=db)

,sid
0,1
1,3


In [54]:
#Nested query:  Ids of students who have taken a course that has been taught 
#at some point by Davidson (needn't have been the semester when Davidson taught)
stmt= ('select distinct sid '+
       'from Takes ' +
       'where cid in (select cid from Teaches natural join Professor'+
       '              where name=\"Davidson\");')
pd.read_sql_query(stmt,con=db)

,sid
0,1
1,3


In [55]:
#(Alternate approach): Ids of students who have taken a course that has been taught 
#at some point by Davidson
stmt= ('select sid '+
       'from (select cid from Teaches natural join Professor where name=\"Davidson\") '+
       '      natural join Takes; ')
pd.read_sql_query(stmt,con=db)

,sid
0,1
1,3


In [56]:
#Nested query: Ids of students who have taken a course that has never been taught 
#by Khanna
stmt= ('select distinct sid '+
       'from Takes ' +
       'where cid not in (select cid from Teaches natural join Professor'+
       '              where name=\"Khanna\");')
pd.read_sql_query(stmt,con=db)

,sid
0,1
1,3


In [57]:
#Correlated subquery: Find ids of students who have taken DB but not Algo
stmt= ('select * '+
       'from (SELECT Takes.sid, subj FROM Takes natural join Course) as T1 ' +
       'where T1.subj=\"DB\" and not exists ' +
       '             (select * ' +
       '              from (SELECT Takes.sid, subj from Takes natural join Course) as T2 ' +
       '              where T2.sid=T1.sid and T2.subj=\"Algo\");')
pd.read_sql_query(stmt,con=db)

,sid,subj
0,3,DB


In [58]:
#Aggregation: Number of professors who have taught each course
stmt= ('select cid, count(*)as num '+
       'from Teaches '+
       'group by cid; ')
pd.read_sql_query(stmt,con=db)

,cid,num
0,521,1
1,550,2
2,677,1
3,700,1


In [59]:
#Aggregation:  Number of different grades expected for each course 
stmt = ('select cid, count(distinct grade) as num ' +
         'from Takes ' +
        'group by cid;')
pd.read_sql_query(stmt,con=db)

,cid,num
0,521,1
1,550,2
2,677,1
3,700,1


In [60]:
#Aggregation with "Having": For each course taught by more than one professor, print the cid and number of professors 
#who have taught it.
stmt= ('select cid, count(*)as num '+
       'from Teaches '+
       'group by cid ' +
       'having count(*)>1 ; ')
pd.read_sql_query(stmt,con=db)

,cid,num
0,550,2


In [61]:
#Setting up an index on name in Student.  Note the use of the cursor to execute the CREATE INDEX statement.  The 
#error message shows that I was successful setting it up the first time I ran it!
c.execute('create index my_index on Student (Name); ')


OperationalError: index my_index already exists